In [5]:
import spacy
import pandas as pd
from afinn import Afinn
import numpy as np

nlp = spacy.load("en_core_web_md")

afinn_df = pd.read_csv("data/AFINN-en-165.tsv", sep="\t", header=None, names=["word", "score"])
#afinn_df
afinn = Afinn()

vad_df = pd.read_csv("data/NRC-VAD-Lexicon.tsv", sep="\t")
#vad_df

In [8]:
afinn_scores = dict(zip(afinn_df["word"], afinn_df["score"]))
#afinn_scores["yummy"]
def afinn_score(text):
    return sum(afinn_scores.get(t.lemma_, 0) for t in nlp(text))

word_valence = dict(zip(vad_df["Word"], vad_df["Valence"]))
word_arousal = dict(zip(vad_df["Word"], vad_df["Arousal"]))
word_dminanc = dict(zip(vad_df["Word"], vad_df["Dominance"]))

def text_valence(text, agg=sum):
    return agg([word_valence.get(t.lemma_, 0) for t in nlp(text)])

def text_arousal(text, agg=sum):
    return agg([word_arousal.get(t.lemma_, 0) for t in nlp(text)])

def text_dminanc(text, agg=sum):
    return agg([word_dminanc.get(t.lemma_, 0) for t in nlp(text)])

In [9]:
tweetDf = pd.read_csv("data/500000tweetshydrated.tsv", sep="\t")
tweetDf["full_text"] = tweetDf["full_text"].astype("string")
#tweetDf.dtypes
#print(tweetDf.full_text)
#tweetDf["wordList"] = wordList(tweetDf["full_text"].tolist())

In [15]:
counting = 0
score = []
for index, row in tweetDf.iterrows():
    #print(row['full_text'], afinn.score(row['full_text']))
    score.append([
        row['Datetime'],
        afinn_score(row['full_text']),
        afinn.score(row['full_text']),
        text_valence(row['full_text']),
        text_arousal(row['full_text']),
        text_dminanc(row['full_text']),
        text_valence(row['full_text'], agg=np.mean),
        text_arousal(row['full_text'], agg=np.mean),
        text_dminanc(row['full_text'], agg=np.mean),
        row['full_text'],
        wordList(row['full_text'])
        ])
    counting+=1
    if (counting % 1000 == 0): print(counting/len(tweetDf.index)*100, '%,', counting, '/', len(tweetDf.index))
    if counting == 100: break

scoredf = pd.DataFrame(score, columns = ['date','AfinnSentiment','AfinnDefaultSentiment','NRCValence','NRCArousal','NRCDominance','NRCValenceMean','NRCArousalMean','NRCDominanceMean','fulltext','lemma'])

In [16]:
scoredf.to_csv("data/500000TweetsHydratedScored.tsv",index = False,sep='\t',line_terminator='\r\n')

In [11]:
def wordList(tText):
    
    doc = nlp(tText)
    ### TOKENIZING ###
    ### REMOVING STOP WORDS ###
    wList = [token for token in doc if not token.is_stop]
    ### NORMALIZING ###
        ### STEMMING ###
        ### LEMMATIZATION ###
    wList = [token.lemma_ for token in wList]
    ### REMOVE PUNCTUATION ###
    punctuations="?:!.,;"
    for word in wList:
        if word in punctuations:
            wList.remove(word)
    ### POS filtering ###
    
    wString = ""
    for each in wList:
        wString += each + " "
    return wString